In [1]:
import requests
from pyspark.sql import SparkSession

# URL for the Iris dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

# Define the column names
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]

# Download the file
response = requests.get(url)
with open("iris.data", "wb") as file:
    file.write(response.content)

# Create Spark session
spark = SparkSession.builder.appName("Iris Dataset").getOrCreate()

# Load the dataset into Spark DataFrame
df = spark.read.csv("iris.data", header=False, inferSchema=True)

# Rename columns
df = df.toDF(*columns)

# Show the first 5 rows
df.show(5)


+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



In [17]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Start Spark session
spark = SparkSession.builder.master("local[*]").appName("IrisModel").getOrCreate()

# Load the Iris dataset with proper column names
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
df = spark.read.csv("iris.data", header=False, inferSchema=True)

# Assign proper column names to the DataFrame
df = df.toDF(*columns)

# Show the first 5 rows
df.show(5)

# Prepare features for machine learning (vectorize the features)
assembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
df = assembler.transform(df)

# Convert the class labels into numerical values (StringIndexer)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="class", outputCol="label")
df = indexer.fit(df).transform(df)

# Split data into training and test sets
train_data, test_data = df.randomSplit([0.7, 0.3], seed=1234)

# Initialize a Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Fit the model
lr_model = lr.fit(train_data)

# Predict on the test data
predictions = lr_model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.2f}")

# Show the first few predictions
predictions.select("sepal_length", "sepal_width", "petal_length", "petal_width", "class", "prediction").show(5)

# You can add additional functionality like filtering, grouping, and aggregating
# For example, grouping by predicted class
predictions.groupBy("prediction").count().show()


+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows

Accuracy: 1.00
+------------+-----------+------------+-----------+-----------+----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|prediction|
+------------+-----------+------------+-----------+-----------+----------+
|         4.3|        3.0|         1.1|        0.1|Iris-setosa|       0.0|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|       0.0|
|        

In [2]:
# Check the schema
df.printSchema()

# Show summary statistics
df.describe().show()

from pyspark.sql.functions import col

# Count the number of missing values in each column
df.select([((col(c).isNull()).cast("int")).alias(c) for c in df.columns]).show()



root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- class: string (nullable = true)

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|         class|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|                150|               150|               150|           150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|          NULL|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|          NULL|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    max|               7.9|                4.4|               6.9|  

In [3]:
# Filter for rows where Sepal Length > 5.0
df_filtered = df.filter(df['sepal_length'] > 5.0)
df_filtered.show()


+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         5.8|        4.0|         1.2|        0.2|Iris-setosa|
|         5.7|        4.4|         1.5|        0.4|Iris-setosa|
|         5.4|        3.9|         1.3|        0.4|Iris-setosa|
|         5.1|        3.5|         1.4|        0.3|Iris-setosa|
|         5.7|        3.8|         1.7|        0.3|Iris-setosa|
|         5.1|        3.8|         1.5|        0.3|Iris-setosa|
|         5.4|        3.4|         1.7|        0.2|Iris-setosa|
|         5.1|        3.7|         1.5|        0.4|Iris-setosa|
|         5.1|        3.3|         1.7|        0.5|Iris-setosa|
|         5.2|        3.5|         1.5| 

In [4]:
# Group by 'class' and count the number of occurrences
df_grouped = df.groupBy('class').count()
df_grouped.show()


+---------------+-----+
|          class|count|
+---------------+-----+
| Iris-virginica|   50|
|    Iris-setosa|   50|
|Iris-versicolor|   50|
+---------------+-----+



In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("test").getOrCreate()
print(spark.version)


3.5.5


In [6]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

# Convert string labels to numeric
indexer = StringIndexer(inputCol="class", outputCol="label")
df_indexed = indexer.fit(df).transform(df)

# Combine features into a single vector
assembler = VectorAssembler(
    inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"],
    outputCol="features"
)
df_prepared = assembler.transform(df_indexed)


In [7]:
train_data, test_data = df_prepared.randomSplit([0.7, 0.3], seed=42)


In [8]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=10)
lr_model = lr.fit(train_data)


In [9]:
predictions = lr_model.transform(test_data)
predictions.select("features", "label", "prediction", "probability").show(5)



from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.2f}")


+-----------------+-----+----------+--------------------+
|         features|label|prediction|         probability|
+-----------------+-----+----------+--------------------+
|[4.4,3.0,1.3,0.2]|  0.0|       0.0|[0.99953289410193...|
|[4.6,3.2,1.4,0.2]|  0.0|       0.0|[0.99947942352750...|
|[4.6,3.6,1.0,0.2]|  0.0|       0.0|[0.99997312562199...|
|[4.7,3.2,1.3,0.2]|  0.0|       0.0|[0.99928771616729...|
|[4.8,3.1,1.6,0.2]|  0.0|       0.0|[0.99636558381973...|
+-----------------+-----+----------+--------------------+
only showing top 5 rows

Accuracy: 0.98


In [14]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

# Initialize Spark session
spark = SparkSession.builder.master("local").appName("IrisModel").getOrCreate()

# Load Iris dataset (use the same dataset you were working with earlier)
data = [
    (5.1, 3.5, 1.4, 0.2, "Iris-setosa"),
    (4.9, 3.0, 1.4, 0.2, "Iris-setosa"),
    (4.7, 3.2, 1.3, 0.2, "Iris-setosa"),
    (4.6, 3.1, 1.5, 0.2, "Iris-setosa"),
    (5.0, 3.6, 1.4, 0.2, "Iris-setosa"),
    (6.7, 3.0, 5.2, 2.3, "Iris-versicolor"),
    (6.3, 3.3, 6.0, 2.5, "Iris-virginica"),
    (5.8, 2.7, 5.1, 1.9, "Iris-virginica"),
    (6.4, 3.2, 4.5, 1.5, "Iris-versicolor"),
    (5.1, 3.8, 1.5, 0.3, "Iris-setosa")
]

# Create a Spark DataFrame
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
df = spark.createDataFrame(data, columns)

# Convert class labels to numeric (0, 1, 2)
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class", outputCol="label")
df = indexer.fit(df).transform(df)

# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
df = assembler.transform(df)

# Split data into training and test sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=1234)

# Initialize Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Train the model
lr_model = lr.fit(train_data)

# Evaluate the model on the test data
test_results = lr_model.evaluate(test_data)
print(f"Test Accuracy: {test_results.accuracy}")

# Predict new data points
new_data = [
    [5.1, 3.5, 1.4, 0.2],
    [6.7, 3.0, 5.2, 2.3],
    [3.7, 3.0, 3.2, 1.3],
    [6.7, 5.0, 1.2, 0.3],
    [5.7, 3.0, 5.2, 0.3]
]

# Convert new data into Spark DataFrame
new_df = spark.createDataFrame([Row(features=Vectors.dense(f)) for f in new_data])

# Make predictions
predictions = lr_model.transform(new_df)

# Show predictions
predictions.select("features", "prediction").show()

# Collect predictions to inspect locally
predicted_rows = predictions.select("features", "prediction").collect()
for row in predicted_rows:
    print(f"Features: {row['features']}, Predicted class: {row['prediction']}")


Py4JJavaError: An error occurred while calling o564.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 32.0 failed 1 times, most recent failure: Lost task 0.0 in stage 32.0 (TID 32) (DESKTOP-83N59GL executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(R

In [15]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("IrisModel") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()


In [16]:
pip show pyspark


Name: pyspark
Version: 3.5.5
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: c:\Users\RickE\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

# Start Spark session
spark = SparkSession.builder.master("local[*]").appName("IrisModel").getOrCreate()

# Load the Iris dataset with proper column names
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
df = spark.read.csv("iris.data", header=False, inferSchema=True)

# Assign proper column names to the DataFrame
df = df.toDF(*columns)

# Show the first 5 rows to inspect the data


# Prepare features for machine learning (vectorize the features)
assembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
df = assembler.transform(df)

# Convert the class labels into numerical values (StringIndexer)
indexer = StringIndexer(inputCol="class", outputCol="label")
df = indexer.fit(df).transform(df)

# Split data into training and test sets
train_data, test_data = df.randomSplit([0.7, 0.3], seed=1234)

# Initialize a Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Fit the model
lr_model = lr.fit(train_data)

# Predict on the test data
predictions = lr_model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.2f}")

# Show the first few predictions
predictions.select("sepal_length", "sepal_width", "petal_length", "petal_width", "class", "prediction").show(5)

# Predict for new rows (5 new observations)
new_data = [
    [5.1, 3.5, 1.4, 0.2],  # Iris-setosa
    [6.7, 3.0, 5.2, 2.3],  # Iris-virginica
    [3.7, 3.0, 3.2, 1.3],  # Iris-versicolor
    [6.7, 5.0, 1.2, 0.3],  # Iris-setosa
    [5.7, 3.0, 5.2, 0.3]   # Iris-virginica
]

# Convert new data into Spark DataFrame (with the same feature structure)
new_df = spark.createDataFrame([Row(sepal_length=float(f[0]), sepal_width=float(f[1]), petal_length=float(f[2]), petal_width=float(f[3])) for f in new_data])

# Assemble features for the new data
new_df = assembler.transform(new_df)

# Make predictions for new data
predictions_new = lr_model.transform(new_df)

# Show the predictions (predicted label for each of the 5 new rows)
predictions_new.select("sepal_length", "sepal_width", "petal_length", "petal_width", "prediction").show()



Accuracy: 1.00
+------------+-----------+------------+-----------+-----------+----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|prediction|
+------------+-----------+------------+-----------+-----------+----------+
|         4.3|        3.0|         1.1|        0.1|Iris-setosa|       0.0|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|       0.0|
|         4.5|        2.3|         1.3|        0.3|Iris-setosa|       0.0|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|       0.0|
|         5.0|        3.0|         1.6|        0.2|Iris-setosa|       0.0|
+------------+-----------+------------+-----------+-----------+----------+
only showing top 5 rows



Py4JJavaError: An error occurred while calling o592.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 150.0 failed 1 times, most recent failure: Lost task 0.0 in stage 150.0 (TID 148) (DESKTOP-83N59GL executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCh

In [4]:
# Collect the predictions to the local machine
predictions_new_list = predictions_new.select("prediction").collect()

# Print the predictions
for row in predictions_new_list:
    print(row['prediction'])


Py4JJavaError: An error occurred while calling o606.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 152.0 failed 1 times, most recent failure: Lost task 0.0 in stage 152.0 (TID 150) (DESKTOP-83N59GL executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrR

In [5]:
# Get a small sample of the predictions
predictions_new_sample = predictions_new.select("prediction").take(5)

# Print the predictions
for row in predictions_new_sample:
    print(row['prediction'])


Py4JJavaError: An error occurred while calling o614.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 153.0 failed 1 times, most recent failure: Lost task 0.0 in stage 153.0 (TID 154) (DESKTOP-83N59GL executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrR